In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


Using TensorFlow backend.


## 什么阻碍了深度神经网络
随着人们年复一年的努力，构建的神经网络也越来越深了，最初的基层，到现在的百多层，为什么要构建越来越深的神经网络，因为越深的神经网络，就能解决越复杂的问题，就能学习到更多不同层次的特征，前面的层会学到基本的特征，例如线条，后面的层可以学到复杂的特征，例如人脸

但是神经网络深度越深，梯度消失就越严重（也会出现梯度爆炸的问题），导致神经网络学得越来越慢，越来越迟钝，所以要使用残差网络来解决这个问题

## 构建残差网络
残差网络是由残差块组成的
下面左图是传统的网络块，右边是残差网络块
其实就是在传统的网络块上添加一条小路，以便让激活值和梯度值可以跳层传播，以避免梯度消失和爆炸
<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> **图 2** </u><font color='purple'>  : 残差块 <br> </center></caption>
实现残差块时，有两种情况：
    1.要跳跃传递的矩阵与目标层矩阵的维度一致时
    2.跳转矩阵与目标层矩阵不一致时（这时就需要变换矩阵的维度）

## 1.维度一致时的残差块
左边的网络层的激活值与小路右边的网络层的激活值的维度是一致的，所以可以直接跳过去。就是说$a^{[l]}$)与$a^{[l+2]}$的维度是一致的。

跳了2层
<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **图 3** </u><font color='purple'>  : . </center></caption>

其实还可以跳更多层，下图就跳了3层: 

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **图 4** </u><font color='purple'>  :</center></caption>

In [3]:
def identity_block(X, f, filters, stage, block):
    """
    实现了图4中展示的残差块
    
    参数：
    X -- 要跳跃的激活值矩阵
    f -- 整型。指示卷积层窗口的大小
    filters -- 整型数组。指示残差块中的卷积层的过滤器的个数
    stage -- 整型。用来辅助给网络层取名
    block -- 字符串。用来辅助给网络层取名
    
    返回：
    X -- 残差块的最终输出矩阵
    """
    
    #取一些名字而已，无关紧要
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    #获取每一个卷积层对于的过滤器的个数
    F1, F2, F3 = filters
    
    #保存输入的激活值，以便后面跳插到后面的网络层中
    X_shortcut = X
    
    #主路中的第一层网络层，就是图4中的第一组绿橙黄小方块
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    #主路中的第二组网络层，就是图4的第二组绿橙黄小方块
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    #主路中的第三组网络层，就是图4的第三组绿橙小方块
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    #这一步就是实现小路的地方。其实就是简单的将前面层的激活值X_shortcut与第三组网络层的输出激活值合并在一起
    #然后将合并的激活值向下传入到激活函数中，进入到后面的神经网络中去
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [4]:
#单元测试
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder('float', [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')
    test.run(tf.global_variables_initializer())
    out = test.run([A],feed_dict={A_prev: X, K.learning_phase(): 0})
    print('out =' + str(out[0][1][2][3]))






out =[ 0.40850517 -0.          0.8418873  -0.          1.6233174   0.2874957 ]


## 维度不一致的时候
<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **图 5** </u><font color='purple'>  :</center></caption>

In [5]:
# 实现图5中的残差块

def convolutional_block(X, f, filters, stage, block, s=2):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)


    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # 在小路上面加上一个卷积层和一个BatchNormalization
    # 卷积层会改变X_shortcut的维度，这样一来，就能与X矩阵合并在一起了
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # 将变维后的X_shortcut与X合并在一起
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


In [6]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [ 0.09018463  1.2348979   0.46822023  0.03671761 -0.          0.65516603]


## 构建一个50层的残差网络
如下图所示，里面的ID BLOCK（红色块）是指维度相同时的残差块，ID BLOCK x3表示有3组这样的残差块。CONV BLOCK（蓝色块）是指维度不同时的残差块
将残差块群范围分成了5个阶段，stage1， stage2...
<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> **图 6** </u><font color='purple'>  : **ResNet-50模型** </center></caption>


In [11]:
def ResNet50(input_shape= (64, 64, 3), classes =6):
    """
    参数：
    input_shape -- 输入的图像矩阵的维度
    classes -- 类别数量
    
    返回：
    model -- 网络模型
    """
    
    #根据输入维度定义一个输入向量
    X_input = Input(input_shape)
    
    #用0填充输入向量的周边
    X = ZeroPadding2D((3, 3))(X_input)
    
    #实现第一阶段，stage1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    #stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    
    # Stage 3 
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    #平均池化层
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    #扁平化激活值矩阵，连上全链接层， softmax层
    X =Flatten()(X)
    X = Dense(classes,activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)
    
    #构建模型
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    
    return model

In [12]:
model = ResNet50(input_shape=(64, 64, 3), classes=6)

## 编译模型

In [15]:
model.compile(optimizer ='adam', loss ='categorical_crossentropy', metrics=['accuracy'])

## 加载数据集

In [16]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [ ]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

## 测试模型精准度

In [ ]:
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

In [ ]:
model = load_model('ResNet50.h5') 
preds = model.evaluate(X_test, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))